In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder,StandardScaler, MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RepeatedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from logging import warning
warning("warnings")

In [2]:
df=pd.read_csv("symptoms_preprocess.csv", index_col=0, encoding="utf-8")
df.head(3)

,0,1,2,3,4,5,6,7,8,9,description
0,105,Panic disorder,Anxiety and nervousness,88,Depression,55,Shortness of breath,40,Depressive or psychotic symptoms,33,'Panic disorder \t \t\t \tPanic disorder is an...
1,106,Vocal cord polyp,Hoarse voice,91,Sore throat,47,Difficulty speaking,27,Cough,27,'Vocal cord polyp \t \t \t\tVocal cord polyp i...
2,107,Turner syndrome,Groin mass,27,Leg pain,27,Hip pain,27,Suprapubic pain,27,'Turner syndrome \t Also known as Gonadal Dysg...


In [8]:
# df[['3', '5', '7', '9']]=df[['3', '5', '7', '9']].astype(str)
df.dtypes

0               int64
1              object
2              object
3              object
4              object
5              object
6              object
7              object
8              object
9              object
description    object
dtype: object

In [30]:
y = LabelEncoder().fit_transform(df['1'])
X = df.drop(columns=['0', '1', 'description'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [31]:
# column_cat = ['2', '4', "6", "8", '3', '5', '7', '9']
column_cat = ['2', '4', "6", "8"]
transfo_cat = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

column_num = ['3', '5', '7', '9']
transfo_num = Pipeline(steps=[('scaling', StandardScaler())])

In [32]:
preparation = ColumnTransformer(
    transformers=[
        ('data_cat', transfo_cat , column_cat),
        ('data_num', transfo_num , column_num)
    ])

In [33]:
cv = RepeatedKFold(n_splits=10, n_repeats=3)
svm= SVC()
rf=RandomForestClassifier()
knn=KNeighborsClassifier()
lr=LogisticRegression()

In [34]:
pipe = Pipeline(steps=[('preparation', preparation),('model',lr)])

In [35]:
print(pipe.get_params().keys())

dict_keys(['memory', 'steps', 'verbose', 'preparation', 'model', 'preparation__n_jobs', 'preparation__remainder', 'preparation__sparse_threshold', 'preparation__transformer_weights', 'preparation__transformers', 'preparation__verbose', 'preparation__verbose_feature_names_out', 'preparation__data_cat', 'preparation__data_num', 'preparation__data_cat__memory', 'preparation__data_cat__steps', 'preparation__data_cat__verbose', 'preparation__data_cat__onehot', 'preparation__data_cat__onehot__categories', 'preparation__data_cat__onehot__drop', 'preparation__data_cat__onehot__dtype', 'preparation__data_cat__onehot__handle_unknown', 'preparation__data_cat__onehot__max_categories', 'preparation__data_cat__onehot__min_frequency', 'preparation__data_cat__onehot__sparse', 'preparation__data_num__memory', 'preparation__data_num__steps', 'preparation__data_num__verbose', 'preparation__data_num__scaling', 'preparation__data_num__scaling__copy', 'preparation__data_num__scaling__with_mean', 'preparatio

In [36]:
parameters = {'model__solver':('newton-cg', 'lbfgs', 'liblinear'),
              'model__penalty':('none', 'l1', 'l2', 'elasticnet')}

In [37]:
grid = GridSearchCV(pipe, parameters, cv = cv)
grid.fit(X_train, y_train)
print('Le meilleur score:', round(grid.best_score_, 2))
print('Les meilleurs paramètres:', grid.best_params_)

c:\Users\Tim_secure\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Tim_secure\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-le

In [ ]:
y_pred = grid.predict(X_test)
print(metrics.classification_report(y_test, y_pred))
print("Coefficient de determination :", round(metrics.f1_score(y_test, y_pred), 2))
cm = metrics.confusion_matrix(y_test, y_pred)
f = sns.heatmap(cm, annot=True, fmt='d', cmap=plt.cm.summer)